# 01_setup_environment.ipynb

This notebook prepares the development environment for the fire detection dissertation project. It performs the following setup tasks:

- Mounts Google Drive for persistent access to datasets, models, and outputs.  
- Creates and verifies the required folder structure in Drive (if not already present).  
- Securely loads a GitHub access token stored in Drive and clones the project repository into the Colab environment.  
- Configures the Git identity for proper commit tracking (required once per session).  

> **Note:** This notebook should be executed at the start of each Colab session.  
> The GitHub token is stored securely in Drive and must **never** be committed to GitHub.


## Step 1: Mount Google Drive

This step mounts Google Drive into the Colab environment at `/content/drive`.  
It provides persistent access to datasets, trained models, results, and the GitHub access token.  
All project files are organized under the directory `/MyDrive/fire-detection-dissertation/`.

Run this cell once at the start of each session. You will be prompted to authorize access to Google Drive.


In [1]:
# Mount Google Drive to enable persistent storage access
from google.colab import drive

drive.mount("/content/drive")

# Project files are located under: /MyDrive/fire-detection-dissertation/


Mounted at /content/drive


## Step 2: Create Project Folder Structure in Drive

This step defines the base directory for the project in Google Drive and creates a consistent folder structure to organize all files.  
The root directory is: `/MyDrive/fire-detection-dissertation/`.

The folders include:  
- `data/raw` → unprocessed datasets (e.g., real D-Fire, PLOS ONE, and synthetic Yunnan/SYN-FIRE).  
- `models` → trained model checkpoints for each phase of experimentation.  
- `figures` → saved outputs including confusion matrices, ROC/PR curves, and Grad-CAM visualizations.  
- `results` → evaluation metrics in JSON format, along with tables and summary outputs.  
- `notebooks` → a backup location for working notebooks stored alongside project data.  

This cell is safe to re-run; it will create any missing folders but will not overwrite existing ones.


In [9]:
# Define the base project directory in Google Drive
base_path = "/content/drive/MyDrive/fire-detection-dissertation"

# Create the standard folder structure if not already present
!mkdir -p {base_path}/data/raw
!mkdir -p {base_path}/models
!mkdir -p {base_path}/figures
!mkdir -p {base_path}/results
!mkdir -p {base_path}/notebooks

# List the full project directory tree to verify setup
!ls -R {base_path}


/content/drive/MyDrive/fire-detection-dissertation:
data  figures  models  notebooks  results  secrets

/content/drive/MyDrive/fire-detection-dissertation/data:
processed  raw

/content/drive/MyDrive/fire-detection-dissertation/data/processed:

/content/drive/MyDrive/fire-detection-dissertation/data/raw:

/content/drive/MyDrive/fire-detection-dissertation/figures:

/content/drive/MyDrive/fire-detection-dissertation/models:

/content/drive/MyDrive/fire-detection-dissertation/notebooks:
01_setup_environment.ipynb

/content/drive/MyDrive/fire-detection-dissertation/results:

/content/drive/MyDrive/fire-detection-dissertation/secrets:
github_token.txt


## Step 3: Load GitHub Token and Clone Repository

This step securely loads the GitHub personal access token from a hidden file stored in Google Drive.  
Using this token, the project repository is cloned into the Colab workspace under `/content/`.

Key points:  
- The token is stored at `secrets/github_token.txt` in Drive and must **never** be pushed to GitHub.  
- The repository is freshly cloned into `/content/` at the start of each session.  
- If a local copy already exists, it is removed before re-cloning to avoid conflicts.  

This process ensures a reproducible setup and guarantees access to the most recent version of the repository.


In [4]:
# Step 3: Load GitHub token from Drive and clone the project repository

# Path to the stored GitHub personal access token
token_path = "/content/drive/MyDrive/fire-detection-dissertation/secrets/github_token.txt"

# Read the token securely
with open(token_path, "r") as f:
    token = f.read().strip()

# GitHub repository details
username = "Misharasapu"  
repo = "fire-detection-dissertation"

# Construct the HTTPS clone URL using the token
clone_url = f"https://{token}@github.com/{username}/{repo}.git"

# Remove any existing local copy to avoid conflicts (safe to re-run)
!rm -rf /content/{repo}

# Clone the repository into the Colab environment
%cd /content
!git clone {clone_url}
%cd {repo}


/content
Cloning into 'fire-detection-dissertation'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), done.
/content/fire-detection-dissertation


## Step 4: Configure Git Identity for Commits

This step sets the Git author information (name and email) so that any commits made from Colab are correctly attributed to your GitHub account.  

Because Colab resets its environment each time it restarts, this configuration must be repeated at the start of every new session.  

If this step is omitted, Git will raise an "author identity unknown" error when attempting to commit.


In [7]:
# Configure Git author identity for commits in this session
!git config --global user.name "Misharasapu"
!git config --global user.email "misharasapu@gmail.com"


## Step 5: Overwrite and Confirm `.gitignore` Rules

This step ensures the repository’s `.gitignore` file contains the necessary rules to exclude non-essential files from version control.  

The following items are excluded:  
- Python cache files and virtual environments.  
- Notebook checkpoints and system/OS files.  
- Model checkpoints (e.g., `.pt`, `.pth`, `.h5`).  
- Large project folders (`data/`, `models/`, `figures/`, `results/`, `secrets/`).  
- The Google Drive mount directory.  

The cell overwrites the `.gitignore` file in the cloned repository and prints its contents for verification.


In [6]:
%cd /content/fire-detection-dissertation

# Step 5: Overwrite .gitignore with project-specific rules
custom_gitignore = """
# Python cache and virtual environments
__pycache__/
*.py[cod]
*.so
.env/
venv/
dist/
build/

# Notebook checkpoints and system files
.ipynb_checkpoints/
.DS_Store

# Output files and logs
*.log
*.out
*.zip

# Model checkpoints
*.pt
*.pth
*.h5

# Project directories to exclude from version control
data/
models/
figures/
results/
secrets/

# Google Drive mount directory
drive/
"""

# Write the updated .gitignore file
gitignore_path = "/content/fire-detection-dissertation/.gitignore"
with open(gitignore_path, "w") as f:
    f.write(custom_gitignore.strip())

# Display the .gitignore contents for verification
print(".gitignore contents:\n")
!cat /content/fire-detection-dissertation/.gitignore


/content/fire-detection-dissertation
✅ .gitignore contents:
# Python cache and virtual envs
__pycache__/
*.py[cod]
*.so
.env/
venv/
dist/
build/

# Notebook checkpoints and system files
.ipynb_checkpoints/
.DS_Store

# Output files and logs
*.log
*.out
*.zip

# Model checkpoints
*.pt
*.pth
*.h5

# Project folders not to track
data/
models/
figures/
results/
secrets/

# Google Drive mount (just in case)
drive/

## Step 6: Create Folder Structure in the GitHub Repository

This step creates the internal folder structure within the cloned GitHub repository.  
These folders organize project notebooks and utility scripts consistently:

- `notebooks/` → experiment and development notebooks.  
- `utils/` → reusable Python modules (e.g., dataset classes, training functions, evaluation helpers).  

This step only needs to be executed once after cloning, but is included in the setup notebook for completeness and reproducibility.


In [11]:
# Step 6: Create the standard folder structure inside the cloned GitHub repository
!mkdir -p /content/fire-detection-dissertation/notebooks
!mkdir -p /content/fire-detection-dissertation/utils


In [ ]:
!cp /content/drive/MyDrive/fire-detection-dissertation/notebooks/01_setup_environment.ipynb /content/fire-detection-dissertation/notebooks/
